<a href="https://colab.research.google.com/github/aismael2022/GVT-V2/blob/main/GVT-V2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install pandas numpy langdetect spacy transformers selenium webdriver-manager
!python -m spacy download en_core_web_sm

# Download your script
!wget https://raw.githubusercontent.com/aismael2022/GVT-V2/main/V4_test.py

# Run the script
%run V4_test.py